**FEATURE EXTRACTION CODE FOR BOTH THE BASELINES**

In [ ]:
import wave
import gc
import librosa
import librosa.display
import IPython
import numpy as np
import tensorflow as tf
import pandas as pd
import sys
from PIL import Image # pip install Pillow
from PIL import ImageOps
from resizeimage import resizeimage
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.client import device_lib

In [ ]:
!pip install python-resize-image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/urbansound8k/UrbanSound8K.csv")

In [ ]:
dataframe['class'].value_counts()

In [ ]:
###VISUALISING AUDIO FILE
y, sr = librosa.load('/content/drive/MyDrive/urbansound8k/fold1/101415-3-0-2.wav)
plt.figure()
plt.subplot(3, 2, 1)
spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
spectrogram = librosa.feature.melspectrogram(S = spectrogram**2)
#### used to display audio file and specto plot
librosa.display.specshow(spectrogram, y_axis='linear')
signal = plt.subplot(2, 2, 3)
signal.plot(y, color="c")

In [ ]:
# Creating the classes directories from Class0 to Class9
specdirectory = "/content/drive/MyDrive/urbansound8k/spectrograms/"
for i in range(0,10):
    Path(specdirectory + "class_" + str(i)).mkdir(parents=True, exist_ok=True)

In [ ]:
def genspectrograms(row):
    classes = row["classID"]
    spect_directory = "class_" + str(classes)
    rawf = row["slice_file_name"][:-4]
    y, sr = librosa.load('/content/drive/MyDrive/urbansound8k/' + "fold" + str(row["fold"]) + "/" + row["slice_file_name"])
    y=librosa.util.normalize(y)
    spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    ## for chroma mfcc and use 
    ## librosa.feature.mfcc(y=y, sr=sr)
    ## librosa.feature.chroma_stft
    spectrogram = librosa.feature.melspectrogram(S = spectrogram**2)
    librosa.display.specshow(spectrogram)
    image = plt.imshow(spectrogram)
    plt.savefig(specdirectory + spect_directory + "/" + rawf + ".png")
    spec=Image.open(specdirectory + spect_directory + "/" + rawf + ".png")
    i=0
    imageSize = image.size
    ## resizing images to average size
    if(imageSize[0]<128 or imageSize[1]<120):
      i=i+1
    invert_im = spec.convert("RGB") 
    invert_im = ImageOps.invert(invert_im)
    imageBox = invert_im.getbbox()
    cropped=spec.crop(imageBox)
    im = resizeimage.resize_cover(cropped, [128, 120])
    im.save(spectrograms_directory + spect_directory + "/" + rawf + ".png") 
    np.save(rawf +str(i)+".npy",spectrogram)
    plt.clf()
    plt.close('all')
    gc.collect()

In [ ]:
#configs
tqdm.pandas()

In [ ]:
dataframe.progress_apply(genspectrograms, axis=1)